In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import AutoModel
from PIL import Image
import os
from pathlib import Path
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np
from tqdm import tqdm

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q "/content/drive/MyDrive/mitosis_detector_BTTAI/dataset_root.zip" -d "/content/dataset_root"

In [ ]:
class Config:
    data_dir = "/content/drive/dataset_root/dataset_root"

    # Training parameters
    batch_size = 256
    num_epochs = 20 # should increase
    learning_rate = 1e-4 * (batch_size / 8)  # = 2e-4
    train_split = 0.8

    # Early stopping
    patience = 10

    # Model parameters
    model_name = "kaiko-ai/midnight"
    embedding_dim = 1536
    num_classes = 2

    # Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Random seed
    seed = 123

NameError: name 'torch' is not defined

In [ ]:
# set seed
torch.manual_seed(Config.seed)
np.random.seed(Config.seed)

In [ ]:
# how to load and process the images
# gets an image, converts to RGB, applies trasnformations and normalizations and returns tensor embeddings
class MitoticDataset(Dataset):
  def __init__(self, image_paths, labels, transform=None):
    self.image_paths = image_paths  # Fixed: was image_path (missing 's')
    self.labels = labels
    self.transform = transform  # Fixed: was transfrom (typo)

  def __len__(self):  # Fixed: indentation - should be at class level, not inside __init__
    return len(self.image_paths)

  def __getitem__(self, idx):  # Fixed: indentation - should be at class level
    image = Image.open(self.image_paths[idx]).convert('RGB')  # Fixed: image_paths
    label = self.labels[idx]  # Fixed: was self.label (missing 's')

    if self.transform:
        image = self.transform(image)

    return image, label

In [ ]:
# data preperation
def load_data(data_dir, train_split=0.8):
    """Load images from folders and create train/val splits"""
    image_paths = []
    labels = []

    # maps class names to numric labels
    class_names = ['1', '2']
    class_to_idx = {'1': 0, '2': 1}

    # Load images from each folder
    for class_name in class_names:

        class_folder = Path(data_dir) / class_name
        if not class_folder.exists():
            raise ValueError(f"Folder not found: {class_folder}")

        class_images = list(class_folder.glob('*.jpeg'))
        print(f"Loading {len(class_images)} images from {class_name}/")

        for img_path in class_images:
            image_paths.append(str(img_path))
            labels.append(class_to_idx[class_name])

    # Convert to numpy arrays
    image_paths = np.array(image_paths)
    labels = np.array(labels)

    # Shuffle because we don't want model to recognize any particular order or take into account the ordering pattern when training
    indices = np.random.permutation(len(image_paths))
    image_paths = image_paths[indices]
    labels = labels[indices]

    # Split into train and validation [TODO: also try cross validation]
    split_idx = int(len(image_paths) * train_split)

    train_paths = image_paths[:split_idx]
    train_labels = labels[:split_idx]

    val_paths = image_paths[split_idx:]
    val_labels = labels[split_idx:]

    print(f"Total images: {len(image_paths)}")
    print(f"Training: {len(train_paths)} images")
    print(f"Validation: {len(val_paths)} images")
    print(f"Class distribution - Non-mitotic: {np.sum(labels == 0)}, Mitotic: {np.sum(labels == 1)}")

    return train_paths, train_labels, val_paths, val_labels, class_names

In [ ]:
# midnight normalizatios: tranfrm 50 x 50 images to 224 x 224 px images
# performs the same trasnformation midnight was trained on
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5),
    std=(0.5, 0.5, 0.5))  # Midnight normalization
])

In [ ]:
# model
class MitoticClassifier(nn.Module):
  def __init__(self, model_name, num_classes, freeze_backbone = True):
    super(MitoticClassifier, self).__init__()

    # load pretrained midnight model
    self.backbone = AutoModel.from_pretrained(model_name)

    # freeze backbone
    if freeze_backbone:
      for param in self.backbone.parameters():
        param.requires_grad = False

    # classification head
    # midnight outputs: [CLS token] + [patch tokens]
    # we'll use CLS token (index 0) + average of patch tokens
    self.classifier = nn.Sequential(
        nn.Linear(Config.embedding_dim * 2, 512), # *2 because we concat CLS + mean patches
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, num_classes)
    )

  def extract_features(self, x):

    """Extract features using Midnight's recommended approach"""
    # Use torch.no_grad() when backbone is frozen, otherwise allow gradients
    if self._is_backbone_frozen():
      with torch.no_grad():
        outputs = self.backbone(x)
    else:
      outputs = self.backbone(x)

    hidden_states = outputs.last_hidden_state

    # Extract CLS token and patch tokens
    cls_token = hidden_states[:, 0, :]  # [batch, embedding_dim]
    patch_tokens = hidden_states[:, 1:, :]  # [batch, num_patches, embedding_dim]

    # Average pool patch tokens
    patch_mean = patch_tokens.mean(dim=1)  # [batch, embedding_dim]

    # Concatenate CLS and patch mean
    features = torch.cat([cls_token, patch_mean], dim=-1)  # [batch, embedding_dim*2]

    return features


  def _is_backbone_frozen(self):
    return not next(self.backbone.parameters()).requires_grad

  def forward(self, x):
    features = self.extract_features(x)
    logits = self.classifier(features)
    return logits

In [ ]:
# training
def train_epoch(model, dataloader, criterion, optimizer, device):
  model.train()
  running_loss = 0.0
  all_preds = []
  all_labels = []

  for images, labels in tqdm(dataloader, desc = "Training"):
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    preds = torch.argmax(outputs, dim = 1)
    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(labels.cpu().numpy())

  epoch_loss = running_loss / len(dataloader)
  epoch_acc = accuracy_score(all_labels, all_preds)

  return epoch_loss, epoch_acc

def validate(model, dataloader, criterion, device):
  model.eval()
  running_loss = 0.0
  all_preds = []
  all_labels = []

  with torch.no_grad():
    for images, labels in tqdm(dataloader, desc = "Validation"):
      images, labels = images.to(device), labels.to(device)

      outputs = model(images)
      loss = criterion(outputs, labels)

      running_loss += loss.item()
      preds = torch.argmax(outputs, dim = 1)
      all_preds.extend(preds.cpu().numpy())
      all_labels.extend(labels.cpu().numpy())

  epoch_loss = running_loss/len(dataloader)
  epoch_acc = accuracy_score(all_labels, all_preds)

  return epoch_loss, epoch_acc, all_preds, all_labels

In [ ]:

# main training loop
def main():
  print(f"Using device: {Config.device}")

  # load data
  train_paths, train_labels, val_paths, val_labels, class_names = load_data(Config.data_dir, Config.train_split)

  # create datasets
  train_dataset = MitoticDataset(train_paths, train_labels, transform = transform)
  val_dataset = MitoticDataset(val_paths, val_labels, transform = transform)

  train_loader = DataLoader(
      train_dataset,
      batch_size = Config.batch_size,
      shuffle = True,
      num_workers = 4,
      pin_memory = True
  )
  val_loader = DataLoader(
      val_dataset,
      batch_size = Config.batch_size,
      shuffle = False,
      num_workers = 4,
      pin_memory = True   #
  )


  print("\nLoading Midnight-12k model...")

  model = MitoticClassifier(
    model_name = Config.model_name,
    num_classes = Config.num_classes,
    freeze_backbone = True # only train classification head
  )

  model = model.to(Config.device)

  # loss and optimizer
  print(f"Model Load Successful")
  print(f"Trainable Parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.classifier.parameters(), lr = Config.learning_rate)

  best_val_acc = 0.0

  # training loop
  print("\nStarting Training...")
  for epoch in range(Config.num_epochs):
    print(f"\nEpoch {epoch+1}/{Config.num_epochs}")
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, Config.device)
    val_loss, val_acc, val_preds, val_labels = validate(model, val_loader, criterion, Config.device)


    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")


    # save best model
    if val_acc > best_val_acc:
      best_val_acc = val_acc
      torch.save(model.state_dict(), "best_model.pth")
      print(f"Saved best model (val acc: {val_acc:.4f})")

  print("\nTraining Complete")


  # final evaluation
  print("Final Evaluation")

  model.load_state_dict(torch.load("best_model.pth"))
  _, val_acc, val_preds, val_labels = validate(model, val_loader, criterion, Config.device)

  print(f"best validation accuracy: {val_acc: .4f}")
  print("\nconfusion matrix:")
  cm = confusion_matrix(val_labels, val_preds)
  print(cm)

  print("\nClassification Report:")
  print(classification_report(val_labels, val_preds, target_names = class_names))

  print(f"Val Acc: {val_acc:.4f}")
  model.eval()


In [ ]:
if __name__ == "__main__":
  main()

Using device: cuda
Loading 12001 images from 1/
Loading 14349 images from 2/
Total images: 26350
Training: 21080 images
Validation: 5270 images
Class distribution - Non-mitotic: 12001, Mitotic: 14349

Loading Midnight-12k model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Model Load Successful
Trainable Parameters: 1,574,402

Starting Training...

Epoch 1/20


Training:   8%|▊         | 7/83 [01:10<12:48, 10.12s/it]


KeyboardInterrupt: 

#**OverPowered Fast Script Starts Here**

In [2]:
!unzip -q "/content/drive/MyDrive/mitosis_detector_BTTAI/BTTAI_Images/dataset_root.zip" -d "/content/dataset_root"

###Feature Extraction

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import AutoModel
from PIL import Image
import os
from pathlib import Path
import numpy as np
from tqdm import tqdm

class Config:
    data_dir = "/content/dataset_root/dataset_root"
    batch_size = 256
    num_epochs = 20
    learning_rate = 1e-4 * (batch_size / 8)
    train_split = 0.8
    patience = 10
    model_name = "kaiko-ai/midnight"
    embedding_dim = 1536
    num_classes = 2
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    seed = 123

class MitoticDataset(Dataset):
  def __init__(self, image_paths, labels, transform=None):
    self.image_paths = image_paths
    self.labels = labels
    self.transform = transform

  def __len__(self):
    return len(self.image_paths)

  def __getitem__(self, idx):
    image = Image.open(self.image_paths[idx]).convert('RGB')
    label = self.labels[idx]
    if self.transform:
        image = self.transform(image)
    return image, label

def load_data(data_dir, train_split=0.8):
    """Load images from folders and create train/val splits"""
    image_paths = []
    labels = []

    class_names = ['1', '2']
    class_to_idx = {'1': 0, '2': 1}

    for class_name in class_names:
        class_folder = Path(data_dir) / class_name
        if not class_folder.exists():
            raise ValueError(f"Folder not found: {class_folder}")

        class_images = list(class_folder.glob('*.jpeg'))
        print(f"Loading {len(class_images)} images from {class_name}/")

        for img_path in class_images:
            image_paths.append(str(img_path))
            labels.append(class_to_idx[class_name])

    image_paths = np.array(image_paths)
    labels = np.array(labels)

    indices = np.random.permutation(len(image_paths))
    image_paths = image_paths[indices]
    labels = labels[indices]

    split_idx = int(len(image_paths) * train_split)

    train_paths = image_paths[:split_idx]
    train_labels = labels[:split_idx]

    val_paths = image_paths[split_idx:]
    val_labels = labels[split_idx:]

    print(f"Total images: {len(image_paths)}")
    print(f"Training: {len(train_paths)} images")
    print(f"Validation: {len(val_paths)} images")
    print(f"Class distribution - Non-mitotic: {np.sum(labels == 0)}, Mitotic: {np.sum(labels == 1)}")

    return train_paths, train_labels, val_paths, val_labels, class_names

class MitoticClassifier(nn.Module):
  def __init__(self, model_name, num_classes, freeze_backbone = True):
    super(MitoticClassifier, self).__init__()

    self.backbone = AutoModel.from_pretrained(model_name)

    if freeze_backbone:
      for param in self.backbone.parameters():
        param.requires_grad = False

    self.classifier = nn.Sequential(
        nn.Linear(Config.embedding_dim * 2, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, num_classes)
    )

  def extract_features(self, x):
    """Extract features using Midnight's recommended approach"""
    if self._is_backbone_frozen():
      with torch.no_grad():
        outputs = self.backbone(x)
    else:
      outputs = self.backbone(x)

    hidden_states = outputs.last_hidden_state


    cls_token = hidden_states[:, 0, :]
    patch_tokens = hidden_states[:, 1:, :]

    patch_mean = patch_tokens.mean(dim=1)

    features = torch.cat([cls_token, patch_mean], dim=-1)

    return features

  def _is_backbone_frozen(self):
    return not next(self.backbone.parameters()).requires_grad

  def forward(self, x):
    features = self.extract_features(x)
    logits = self.classifier(features)
    return logits

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

def extract_and_save(loader, save_dir, split_name):
    all_features = []
    all_labels = []

    print(f"Extracting features for {split_name}...")
    with torch.no_grad():
        for images, labels in tqdm(loader, desc=f"Extracting {split_name}"):
            images = images.to(Config.device)

            features = model.extract_features(images)

            all_features.append(features.cpu())
            all_labels.append(labels.cpu())

    final_features = torch.cat(all_features, dim=0)
    final_labels = torch.cat(all_labels, dim=0)

    print(f"Finished. Feature tensor shape: {final_features.shape}")
    print(f"Finished. Label tensor shape: {final_labels.shape}")

    torch.save(final_features, os.path.join(save_dir, f"{split_name}_features.pt"))
    torch.save(final_labels, os.path.join(save_dir, f"{split_name}_labels.pt"))

if __name__ == "__main__":
    print(f"Using device: {Config.device}")
    torch.manual_seed(Config.seed)
    np.random.seed(Config.seed)

    os.makedirs("features", exist_ok=True)

    # Load data paths
    train_paths, train_labels, val_paths, val_labels, _ = load_data(Config.data_dir, Config.train_split)

    # Create datasets
    train_dataset = MitoticDataset(train_paths, train_labels, transform=transform)
    val_dataset = MitoticDataset(val_paths, val_labels, transform=transform)

    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=Config.batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=Config.batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )

    # Load Model
    print("Loading Midnight-12k model for feature extraction...")
    model = MitoticClassifier(
        model_name=Config.model_name,
        num_classes=Config.num_classes,
        freeze_backbone=True
    )
    model = model.to(Config.device)
    model.eval()

    # Run the extraction for both train and validation sets
    extract_and_save(train_loader, "features", "train")
    extract_and_save(val_loader, "features", "val")

    print("\nFeature extraction complete!")


Using device: cuda
Loading 11937 images from 1/
Loading 14349 images from 2/
Total images: 26286
Training: 21028 images
Validation: 5258 images
Class distribution - Non-mitotic: 11937, Mitotic: 14349
Loading Midnight-12k model for feature extraction...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Extracting features for train...


Extracting train: 100%|██████████| 83/83 [11:40<00:00,  8.44s/it]


Finished. Feature tensor shape: torch.Size([21028, 3072])
Finished. Label tensor shape: torch.Size([21028])
Extracting features for val...


Extracting val: 100%|██████████| 21/21 [02:55<00:00,  8.36s/it]

Finished. Feature tensor shape: torch.Size([5258, 3072])
Finished. Label tensor shape: torch.Size([5258])

Feature extraction complete!


###Classification Head Training

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np
from tqdm import tqdm
import os
import glob
from pathlib import Path
import pandas as pd

class Config:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    TRAIN_FEATURES_PATH = "features/train_features.pt"
    TRAIN_LABELS_PATH = "features/train_labels.pt"
    VAL_FEATURES_PATH = "features/val_features.pt"
    VAL_LABELS_PATH = "features/val_labels.pt"

    data_dir = "/content/dataset_root/dataset_root"
    train_split = 0.8

    batch_size = 256
    num_epochs = 100
    base_learning_rate = 1e-4 * (batch_size / 8)

    embedding_dim = 1536
    num_classes = 2

    label_smoothing = 0.1

    seed = 123

class FeatureDataset(Dataset):
    def __init__(self, feature_path, label_path):
        print(f"Loading features from {feature_path} into RAM...")
        self.features = torch.load(feature_path)
        self.labels = torch.load(label_path).long()
        print(f"Done. Loaded {len(self.features)} samples.")

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

def get_classifier_head():
    classifier_head = nn.Sequential(
        nn.Linear(Config.embedding_dim * 2, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, Config.num_classes)
    ).to(Config.device)
    return classifier_head

def load_data(data_dir=Config.data_dir, train_split=Config.train_split):
    image_paths = []
    labels = []

    class_names = ['1', '2']
    class_to_idx = {'1': 0, '2': 1}

    for class_name in class_names:
        class_folder = Path(data_dir) / class_name
        if not class_folder.exists():
            raise ValueError(f"Folder not found: {class_folder}")

        class_images = list(class_folder.glob('*.jpeg'))

        for img_path in class_images:
            image_paths.append(str(img_path))
            labels.append(class_to_idx[class_name])

    image_paths = np.array(image_paths)
    labels = np.array(labels)

    indices = np.random.permutation(len(image_paths))
    image_paths = image_paths[indices]
    labels = labels[indices]

    split_idx = int(len(image_paths) * train_split)

    train_paths = image_paths[:split_idx]
    train_labels = labels[:split_idx]

    val_paths = image_paths[split_idx:]
    val_labels = labels[split_idx:]

    return train_paths, train_labels, val_paths, val_labels, class_names


def train_epoch_fast(model, dataloader, criterion, optimizer, scheduler, device):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    for features, labels in tqdm(dataloader, desc="Training"):
        features, labels = features.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        scheduler.step()

        running_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    epoch_loss = running_loss / len(dataloader)
    epoch_acc = accuracy_score(all_labels, all_preds)
    return epoch_loss, epoch_acc

def validate_fast(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for features, labels in tqdm(dataloader, desc="Validation"):
            features, labels = features.to(device), labels.to(device)

            outputs = model(features)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    epoch_loss = running_loss / len(dataloader)
    epoch_acc = accuracy_score(all_labels, all_preds)
    return epoch_loss, epoch_acc, all_preds, all_labels

def main():
    print(f"Using device: {Config.device}")
    torch.manual_seed(Config.seed)
    np.random.seed(Config.seed)

    train_feat_dataset = FeatureDataset(Config.TRAIN_FEATURES_PATH, Config.TRAIN_LABELS_PATH)
    val_feat_dataset = FeatureDataset(Config.VAL_FEATURES_PATH, Config.VAL_LABELS_PATH)

    print("Calculating class weights for sampler...")
    train_labels = train_feat_dataset.labels.numpy()
    class_counts = np.bincount(train_labels)
    class_weights = 1. / torch.tensor(class_counts, dtype=torch.float)
    samples_weight = class_weights[train_labels]

    sampler = WeightedRandomSampler(
        weights=samples_weight,
        num_samples=len(samples_weight),
        replacement=True
    )
    print("Sampler created.")

    train_loader = DataLoader(
        train_feat_dataset,
        batch_size=Config.batch_size,
        sampler=sampler,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_feat_dataset,
        batch_size=Config.batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

    model = get_classifier_head()
    print(f"Classifier head created. Trainable Parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

    criterion = nn.CrossEntropyLoss(label_smoothing=Config.label_smoothing)

    optimizer = optim.Adam(model.parameters(), lr=Config.base_learning_rate)

    from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
    num_steps_per_epoch = len(train_loader)

    scheduler = CosineAnnealingWarmRestarts(
        optimizer,
        T_0=num_steps_per_epoch,
        eta_min=1e-6
    )

    best_val_acc = 0.0

    print("\nStarting FAST Training...")
    for epoch in range(Config.num_epochs):
        print(f"\nEpoch {epoch+1}/{Config.num_epochs}")

        train_loss, train_acc = train_epoch_fast(model, train_loader, criterion, optimizer, scheduler, Config.device)

        val_loss, val_acc, val_preds, val_labels = validate_fast(model, val_loader, criterion, Config.device)

        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        print(f"Current LR: {optimizer.param_groups[0]['lr']:.6f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_classifier_head.pth")
            print(f"Saved best model (val acc: {val_acc:.4f})")

    print("\nTraining Complete")

    print("Final Evaluation")
    model.load_state_dict(torch.load("best_classifier_head.pth"))

    _, val_acc, val_preds, val_labels = validate_fast(model, val_loader, criterion, Config.device)

    print(f"Best validation accuracy: {val_acc:.4f}")

    print("\n--- Error Analysis: Loading File Paths ---")
    np.random.seed(Config.seed)

    _, _, val_paths, _, class_names_str = load_data()
    val_paths = np.array(val_paths)

    val_preds_np = np.array(val_preds)
    val_labels_np = np.array(val_labels)

    wrong_mask = (val_preds_np != val_labels_np)

    wrong_files = val_paths[wrong_mask]
    wrong_predictions = val_preds_np[wrong_mask]
    true_labels = val_labels_np[wrong_mask]

    print(f"Total incorrect predictions: {len(wrong_files)}")

    fp_mask = (wrong_predictions == 1) & (true_labels == 0)
    fp_files = wrong_files[fp_mask]

    FP_types = []
    FN_types = []
    IMG_CSV = "/content/drive/MyDrive/mitosis_detector_BTTAI/images.csv"
    df = pd.read_csv(IMG_CSV)

    print(f"\n--- False Positives (Predicted Mitotic, was Non-Mitotic): {len(fp_files)} ---")
    for f_fp in fp_files[:min(500, len(fp_files))]: # changed from 10
        filename = os.path.basename(f_fp)
        img_name_fp = filename.partition(".")[0]

        id_fp = img_name_fp.partition("_")[0]

        if len(id_fp) == 2:
            id_fp_f = "0" + id_fp
        elif len(id_fp) == 1:
            id_fp_f = "00" + id_fp
        else:
          id_fp_f = id_fp

        cond_fp = df['file_name'] == id_fp_f + ".tiff"
        idx_fp = df.index[cond_fp]
        name_fp = df.loc[idx_fp, 'tumor_type'].iloc[0]
        FP_types.append((img_name_fp, name_fp))

    fn_mask = (wrong_predictions == 0) & (true_labels == 1)
    fn_files = wrong_files[fn_mask]
    print(f"\n--- False Negatives (Predicted Non-Mitotic, was Mitotic): {len(fn_files)} ---")
    for f_fn in fn_files[:min(500, len(fn_files))]: #changed from 10
        filename = os.path.basename(f_fn)
        img_name_fn = filename.partition(".")[0]

        id_fn = img_name_fn.partition("_")[0]

        if len(id_fn) == 2:
            id_fn_f = "0" + id_fn
        elif len(id_fn) == 1:
            id_fn_f = "00" + id_fn
        else:
            id_fn_f = id_fn

        cond_fn = df['file_name'] == id_fn_f + ".tiff"
        idx_fn = df.index[cond_fn]
        name_fn = df.loc[idx_fn, 'tumor_type'].iloc[0]
        FN_types.append((img_name_fn, name_fn))

    from collections import defaultdict
    from itertools import chain

    group_by_image = defaultdict(list)


    def print_groups(samples):
      #images_list = []

      groups = {}

      for img_id, category in samples:
          groups.setdefault(category, []).append(img_id)

      for category, ids in groups.items():
          print(f"{category} ({len(ids)} patches): {ids}")
          groups=defaultdict(list)
          for id_ in ids:
              prefix, subset = id_.split("_", 1)
              groups[prefix].append(id_)
          #print(groups)
          print(f"Number of images in {category}: " + str(len(groups)))


          #images_list.append(ids)
      #print("list of images:")
      #print(images_list)
      #print(len(images_list))
      #images_list_flat = list(chain.from_iterable(images_list))
      #groups = defaultdict(list)

      #for id_ in images_list_flat:
          #prefix, subset = id_.split("_", 1)
          #groups[prefix].append(id_)
      #print(groups)

      #print("Number of images: " + str(len(groups)))


    print("False Negatives:")
    print_groups(FN_types)
    print("\nFalse Positives:")
    print_groups(FP_types)

    print("\nConfusion Matrix:")
    cm = confusion_matrix(val_labels, val_preds)
    print(cm)

    print("\nClassification Report:")
    class_names = ['1 (Non-mitotic)', '2 (Mitotic)']
    print(classification_report(val_labels, val_preds, target_names=class_names))

if __name__ == "__main__":
    main()

Using device: cuda
Loading features from features/train_features.pt into RAM...
Done. Loaded 21028 samples.
Loading features from features/val_features.pt into RAM...
Done. Loaded 5258 samples.
Calculating class weights for sampler...
Sampler created.
Classifier head created. Trainable Parameters: 1,574,402

Starting FAST Training...

Epoch 1/100


Validation: 100%|██████████| 21/21 [00:00<00:00, 99.76it/s]


Train Loss: 0.5911 | Train Acc: 0.7227
Val Loss: 0.5305 | Val Acc: 0.7708
Current LR: 0.003200
Saved best model (val acc: 0.7708)

Epoch 2/100


Validation: 100%|██████████| 21/21 [00:00<00:00, 100.58it/s]


Train Loss: 0.5162 | Train Acc: 0.7813
Val Loss: 0.5073 | Val Acc: 0.7868
Current LR: 0.003200
Saved best model (val acc: 0.7868)

Epoch 3/100


...

Train Loss: 0.3844 | Train Acc: 0.8883
Val Loss: 0.4718 | Val Acc: 0.8227
Current LR: 0.003200

Epoch 100/100


Validation: 100%|██████████| 21/21 [00:00<00:00, 99.01it/s]


Train Loss: 0.3909 | Train Acc: 0.8819
Val Loss: 0.4672 | Val Acc: 0.8224
Current LR: 0.003200

Training Complete
Final Evaluation


Validation: 100%|██████████| 21/21 [00:00<00:00, 94.86it/s]


Best validation accuracy: 0.8269

--- Error Analysis: Loading File Paths ---
Total incorrect predictions: 910

--- False Positives (Predicted Mitotic, was Non-Mitotic): 432 ---

--- False Negatives (Predicted Non-Mitotic, was Mitotic): 478 ---
False Negatives:
canine lymphosarcoma (174 patches): ['246_139', '258_69', '283_192', '288_227', '299_106', '299_150', '267_3', '270_136', '293_115', '250_214', '268_99', '245_53', '299_93', '258_95', '250_135', '258_118', '256_58', '268_177', '299_166', '298_135', '267_37', '272_182', '277_58', '284_22', '251_70', '263_111', '296_23', '264_14', '267_75', '275_16', '285_68', '299_164', '288_220', '261_64', '275_31', '260_64', '288_178', '245_120', '286_57', '262_136', '251_87', '250_120', '251_54', '250_6', '251_73', '264_11', '271_46', '273_121', '279_21', '250_172', '259_80', '281_21', '271_201', '271_166', '264_9', '287_37', '283_106', '284_57', '254_73', '295_51', '268_149', '249_110', '299_25', '269_162', '280_155', '267_126', '261_4', '262_

In [ ]:
import os
import shutil
from pathlib import Path


DRIVE_BASE_PATH = "/content/drive/MyDrive/"
FP_FOLDER_NAME = "False Positives"
FN_FOLDER_NAME = "False Negatives"


fp_target_path = os.path.join(DRIVE_BASE_PATH, FP_FOLDER_NAME)
fn_target_path = os.path.join(DRIVE_BASE_PATH, FN_FOLDER_NAME)

os.makedirs(fp_target_path, exist_ok=True)
os.makedirs(fn_target_path, exist_ok=True)

print(f"Created or found folder: {fp_target_path}")
print(f"Created or found folder: {fn_target_path}")

fp_files_to_copy = [
    "/content/dataset_root/dataset_root/1/498_43.jpeg",
    "/content/dataset_root/dataset_root/1/298_95.jpeg",
    "/content/dataset_root/dataset_root/1/34_47.jpeg",
    "/content/dataset_root/dataset_root/1/542_109.jpeg",
    "/content/dataset_root/dataset_root/1/356_5.jpeg",
    "/content/dataset_root/dataset_root/1/346_207.jpeg",
    "/content/dataset_root/dataset_root/1/539_21.jpeg",
    "/content/dataset_root/dataset_root/1/504_34.jpeg",
    "/content/dataset_root/dataset_root/1/524_12.jpeg",
    "/content/dataset_root/dataset_root/1/253_85.jpeg",
]

fn_files_to_copy = [
    "/content/dataset_root/dataset_root/2/131_34.jpeg",
    "/content/dataset_root/dataset_root/2/252_59.jpeg",
    "/content/dataset_root/dataset_root/2/40_17.jpeg",
    "/content/dataset_root/dataset_root/2/314_11.jpeg",
    "/content/dataset_root/dataset_root/2/474_1.jpeg",
    "/content/dataset_root/dataset_root/2/325_110.jpeg",
    "/content/dataset_root/dataset_root/2/452_28.jpeg",
    "/content/dataset_root/dataset_root/2/395_78.jpeg",
    "/content/dataset_root/dataset_root/2/256_39.jpeg",
    "/content/dataset_root/dataset_root/2/45_7.jpeg",
]


print(f"\nCopying {len(fp_files_to_copy)} False Positive images...")
for src_path in fp_files_to_copy:
    if os.path.exists(src_path):
        file_name = Path(src_path).name
        dest_path = os.path.join(fp_target_path, file_name)
        try:
            shutil.copy2(src_path, dest_path)
        except Exception as e:
            print(f"Error copying {src_path} to {dest_path}: {e}")
    else:
        print(f"Warning: Source file not found - {src_path}")

print(f"\nCopying {len(fn_files_to_copy)} False Negative images...")
for src_path in fn_files_to_copy:
    if os.path.exists(src_path):
        file_name = Path(src_path).name
        dest_path = os.path.join(fn_target_path, file_name)
        try:
            shutil.copy2(src_path, dest_path)
        except Exception as e:
            print(f"Error copying {src_path} to {dest_path}: {e}")
    else:
        print(f"Warning: Source file not found - {src_path}")

print("\nFinished copying files.")

Created or found folder: /content/drive/MyDrive/False Positives
Created or found folder: /content/drive/MyDrive/False Negatives

Copying 10 False Positive images...

Copying 10 False Negative images...

Finished copying files.
